In [39]:
import tabula
import os

import pandas as pd

In [22]:
files = os.listdir(../data/historico-facturas-pdf)

In [23]:
files = [i for i in files if ".DS" not in i]

In [24]:
files

['ticket-302316.pdf',
 'ticket-125935.pdf',
 'ticket-404271.pdf',
 'ticket-466635.pdf',
 'ticket-73955.pdf',
 'ticket-185199.pdf',
 'ticket-278125.pdf',
 'ticket-935378.pdf',
 'ticket-208575.pdf',
 'ticket-996926.pdf',
 'ticket-546427.pdf',
 'ticket-616519.pdf',
 'ticket-803441.pdf',
 'ticket-577760.pdf',
 'ticket-107492.pdf',
 'ticket-470822.pdf',
 'ticket-95302.pdf',
 'ticket-209834.pdf',
 'ticket-986079.pdf',
 'ticket-156155.pdf',
 'ticket-61858.pdf',
 'ticket-337881.pdf',
 'ticket-686714.pdf',
 'ticket-101295.pdf',
 'ticket-786946.pdf',
 'ticket-134160.pdf',
 'ticket-207666.pdf']

In [26]:
uno = files[0]
uno

'ticket-302316.pdf'

In [48]:
tabula.read_pdf("ticket-302316.pdf", pages="all", pandas_options={'header': None})[0]

,0,1,2
0,Jamón serrano Incarlopsa lonchas,1,"3,49 €"
1,Queso fresco de cabra Hacendado,1,"2,20 €"
2,Queso rallado 4 quesos Hacendado,1,"1,59 €"
3,Perlas de mozzarella fresca Hacendado,1,"1,70 €"
4,Mozzarella di bufala campana Hacendado,1,"1,70 €"
5,Refresco Coca-Cola Zero azúcar,1,"7,56 €"
6,Salchichón extra pimienta Hacendado finas lonchas,1,"1,38 €"
7,Mortadela italiana Hacendado finas lonchas,1,"1,69 €"
8,Salfumant Bosque Verde,1,"0,75 €"
9,Dentífrico triple acción Colgate con flúor,2,"3,50 €"


In [ ]:
df

In [34]:
tables

NameError: name 'tables' is not defined